In [12]:
import csv
import pandas as pd 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import numpy as np

In [21]:
df = pd.read_csv("C:\\Users\\Connor\\Desktop\\Coding\\nft_market_a\\scripts_\\twitter_nft_timeline.csv")
df.head()

,Tweet ID,Username,User ID,Tweet Date,Text
0,1658216917662834702,sara_b022,1528319102141485062,2023-05-15 21:04:56+00:00,@BlueSparrowfans @queencryptooo #BlueSparrow\n...
1,1658216916412932129,yarikdoren,1616809072992583682,2023-05-15 21:04:55+00:00,"🏴‍☠️ Ahoy, treasure hunters! I earned my pira..."
2,1658216906199801867,SeizeTheMonkey,1398003143791759368,2023-05-15 21:04:53+00:00,NFT projects’ social media platform comps:\n\n...
3,1658216899765739546,ForestKnight_io,734818764924063745,2023-05-15 21:04:51+00:00,1st place: Rare NFT weapon for Forest knight o...
4,1658216891989499926,Joax_NFT,1531335738071339008,2023-05-15 21:04:49+00:00,@Zeck_Sol @ARAY_nft @SoAegis @GreediesGood @0x...


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Tweet ID    600 non-null    int64 
 1   Username    600 non-null    object
 2   User ID     600 non-null    int64 
 3   Tweet Date  600 non-null    object
 4   Text        600 non-null    object
dtypes: int64(2), object(3)
memory usage: 23.6+ KB


In [23]:
# Function to clean words up in data
def process_text(tweet):
    tweet_words = []
    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1: # Converts their username to '@user'
            word = '@user'
        elif word.startswith('http'): # Converts website address to 'http'
            word = "http"
        tweet_words.append(word)
    tweet_proc = " ".join(tweet_words)
    return tweet_proc

In [24]:
df.head()

,Tweet ID,Username,User ID,Tweet Date,Text
0,1658216917662834702,sara_b022,1528319102141485062,2023-05-15 21:04:56+00:00,@BlueSparrowfans @queencryptooo #BlueSparrow\n...
1,1658216916412932129,yarikdoren,1616809072992583682,2023-05-15 21:04:55+00:00,"🏴‍☠️ Ahoy, treasure hunters! I earned my pira..."
2,1658216906199801867,SeizeTheMonkey,1398003143791759368,2023-05-15 21:04:53+00:00,NFT projects’ social media platform comps:\n\n...
3,1658216899765739546,ForestKnight_io,734818764924063745,2023-05-15 21:04:51+00:00,1st place: Rare NFT weapon for Forest knight o...
4,1658216891989499926,Joax_NFT,1531335738071339008,2023-05-15 21:04:49+00:00,@Zeck_Sol @ARAY_nft @SoAegis @GreediesGood @0x...


In [25]:
df['processed_text'] = df['Text'].apply(process_text)

print(df['processed_text'])

0      @user @user #BlueSparrow\n#BlueSparrowToken\n#...
1      🏴‍☠️  Ahoy, treasure hunters! I earned my pira...
2      NFT projects’ social media platform comps:\n\n...
3      1st place: Rare NFT weapon for Forest knight o...
4      @user @user @user @user @user Bro... Lets get it!
                             ...                        
595    🪂Airdrop for NFT holders VoldiAI !!\n💰Owners o...
596                        @user Is already droppinh sir
597    @user @user @user @user @user Hey  👋 let's col...
598    The first Bitcoin purchase ever was for a JPEG...
599    @user @user @user @user @user @user @user @use...
Name: processed_text, Length: 600, dtype: object


In [26]:
# Load Model
# Specify the model
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

# Load model
model = AutoModelForSequenceClassification.from_pretrained(roberta)

# Load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained(roberta)

# Define labels
labels = ['Negative', 'Neutral', 'Positive']

In [27]:
# Sentiment analysis

# Initialise a variable to store the total scores
total_scores = np.zeros((1, len(labels)))

for tweet_proc in df['processed_text']:
    # Tokenize the tweet and return PyTorch tensors
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
    
    # Feed the encoded input IDs and attention masks to get output logits
    output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
    
    # Detach the logits from the computational graph and convert them into a NumPy array
    logits = output.logits.detach().numpy()
    
    # Apply the softmax function to the logits to obtain probability scores for each category
    scores = np.exp(logits) / np.sum(np.exp(logits), axis=1, keepdims=True)

    
    # Add the scores to the total scores
    total_scores += scores
    
# Calculate the average scores 
average_scores = total_scores / len(df['processed_text'])

# Find the index of the highest average score
max_index = np.argmax(average_scores)

print(f"{labels[max_index]}: {average_scores[0][max_index]}")

Positive: 0.4583768867146379
